In [1]:
import json
import numpy as np
from pancreas import *
from odeclass import *

In [2]:
def MVP(self, u, d):
    """
    Solves dx = f(x, u, d)

    Parameters
    ----------
    u : int or float 
        Insulin injection rate.
    d : int or float 
        Meal ingestion rate.
    
    Returns
    -------
    dx : numpy array
        Solution to system of differential equations. 
    """
    dD1 = d - self.D1/self.taum
    dD2 = (self.D1 - self.D2)/self.taum
    dIsc = u/(self.tau1 * self.C1) - self.Isc/self.tau1
    dIp = (self.Isc - self.Ip)/self.tau2
    dIeff = -self.p2 * self.Ieff + self.p2 * self.S1 * self.Ip
    dG = - (self.gezi + self.Ieff) * self.G + self.egp0 + 1000 * self.D2 / (self.Vg * self.taum)
    dGsc = (self.G - self.Gsc) / self.timestep

    dx = np.array([dD1, dD2, dIsc, dIp, dIeff, dG, dGsc])
    return dx


def EHM(self, d, uI, uP, HR = ):
    """
    Solves dx = f(x, u, d)

    Parameters
    ----------
    u : int or float 
        Insulin injection rate.
    d : int or float 
        Meal ingestion rate.
    
    Returns
    -------
    dx : numpy array
        Solution to system of differential equations. 
    """
    G = self.Q1/self.VG
    RA = self.f*self.D2/self.TauD
    D = 1000 * d/self.MwG

    if HR is None:
        HRmax = 220 - self.age
        HR = self.HRR * (HRmax - self.HR0) + self.HR0
    fE1x = self.E1/(self.a * self.HR0)**self.n
    fE1 = fE1x/(1+fE1x)
    F01c = min(self.F01, self.F01 * G / 4.5)
    FR = max(0.003 * (G - 9) * self.VG, 0) 

    dx = []
    dx.append((G - self.G)/self.TauIG)
    dx.append(-F01c - FR - self.x1 * self.Q1 + self.k12 * self.Q2 + RA + self.EGP0 * (1 - self.x3) \
        + self.Kglu * self.VG * self.Z2 - self.alpha * self.E2**2 * self.x1 * self.Q1)
    dx.append(self.x1 * self.Q1 - (self.k12 + self.x2) * self.Q2 + self.alpha * self.E2**2 * self.x1 * self.Q1 \
        - self.alpha * self.E2**2 * self.x2 * self.Q2 - self.beta * self.E1 / self.HR0)

    dx.append(uI - self.S1 / self.TauS)
    dx.append((self.S1 - self.S2)/self.TauS)
    dx.append(self.S2 / (self.VI * self.TauS) - self.ke * self.I)
    dx.append(self.kb1 * self.I - self.ka1 * self.x1)
    dx.append(self.kb2 * self.I - self.ka2 * self.x2)
    dx.append(self.kb3 * self.I - self.ka3 * self.x3)
    dx.append(self.AG * D - self.D1 / self.TauD)
    dx.append((self.D1 - self.D2)/self.TauD)
    dx.append(uG - self.Z1 / self.Tauglu)
    dx.append((self.Z1 - self.Z2)/self.Tauglu)
    dx.append((HR - self.HR0 - self.E1)/self.TauHR)
    dx.append(-(fE1/self.Tauin - 1/self.TE) * self.E2 + fE1 * self.TE /(self.c1 + self.c2))
    dx.append((self.c1 * fE1 + self.c2 - self.TE)/self.Tauex)
    return np.array(dx)

In [3]:
class Healthy(ODE):
    def __init__(self, model = "EHM", **kwargs):
        self.model = model.upper()
        
        defaults = {} #tomt dictionary 
        with open('config.json', 'r') as f:
            data = json.load(f) #læs json-fil
        defaults.update(data["general"]) #tilføj "general" til dictionary(defaults)
        defaults.update(data[self.model]) #tilføj modelegenskaberne til dictionary(defaults)
        defaults.update(kwargs) #tilføj keywordarguments til dictionary(defaults)

        super().__init__(defaults)
        if self.model == "MVP":
            self.f_func = lambda *args: MVP(self,*args) #caller MVP-modellen
        else:
            self.f_func = lambda *args: EHM(self,*args) #caller EHM-modellen

        self.pancreas = PKPM(timestep=self.timestep, Gbar=self.Gbar)

    def simulate(self, ds, ):
        """
        Simulates patient.

        Parameters
        ----------
        ds : numpy array
            Ingestion rate
        u_func : Default = None, int, float, numpy array, list or "PID"
            Specifies insulin injection rate.
            If None; uses steady state insulin rate.
            If "PID"; uses PID controller.
        
        Returns
        -------
        states : numpy array
            State vector in each time step
        u_list : numpy array
            Insulin injection rate for each time step
        """
        res = np.empty((len(ds)+1, len(self.state_keys)))
        info = dict()
        for i in self.state_keys:
            info[i]=np.empty(len(ds)+1)
            info[i][0]=getattr(self,i)
        res[0, :] = self.get_state()
        inp = 0
        u = self.us
        if isinstance(u_func, (np.ndarray, list)):
            u = u_func[0]
            inp = 1
            def get_u(inp):
                u = u_func[inp]
                return u, inp+1

        elif u_func == "PID":
            inp = (0, self.G)
            def get_u(inp):
                I, y_prev = inp
                u, I = self.PID_controller(I, self.Gsc, y_prev)
                return u, (I, self.G)

        elif isinstance(u_func, (float, int)):
            u = u_func
            def get_u(inp):
                return u_func, inp

        else:
            def get_u(inp):
                return self.us, inp

        u_list = [u]

        for i,d in enumerate(ds[:-1]):
            dx = self.f_func(u, d)
            self.euler_step(dx)     
            res[i+1,:] = self.get_state()
            for k in self.state_keys:
                info[k][i+1]=getattr(self,k)
            u, inp = get_u(inp)
            u_list.append(u)
            info["u"][i+1]=u
        dx = self.f_func(u, d)
        self.euler_step(dx)     
        res[i+2, :] = self.get_state()
        for k in self.state_keys:
            info[k][i+2]=getattr(self,k)
        info["pens"]=self.glucose_penalty(info["G"])
        info["u"]=np.array(u_list)
        
        return np.array(res), u_list, info
p = Healthy()
p.pancreas.eval(3)
p.pancreas.get_ISR(3)

10180922.4

In [4]:
class TD1:
    def __init__(self, model ="EHM", **kwargs):
        self.model = model.upper()
        
        defaults = {} #tomt dictionary 
        with open('config.json', 'r') as f:
            data = json.load(f) #læs json-fil
        defaults.updata(data["general"]) #tilføj "general" til dictionary(defaults)
        defaults.update(data[self.model]) #tilføj modelegenskaberne til dictionary(defaults)
        defaults.update(kwargs) #tilføj keywordarguments til dictionary(defaults)

        if self.model == "MVP":
            self.f_func = lambda *args: MVP(self,*args) #caller MVP-modellen
        else:
            self.f_func = lambda *args: EHM(self,*args) #caller EHM-modellen

    def PID_controller(self, I, y, y_prev):
        """
        :input I: Integral term
        :input Gbar: Glucose concentration target
        :input y: Current blood glucose
        :input y_prev: Previous blood glucose
        :input us: Insulin steady state
        Tuning parameters
        :input Kp: Range 0-0.5
        :input Ti: 100-1000 minutes
        :input Td : 0-30 min
        """
        ek = y - self.Gbar
        Pk = self.Kp * ek
        Ki = self.Kp * self.timestep / self.Ti
        Ikp1 = I + Ki * ek
        Kd = self.Kp * self.Td / self.timestep
        Dk = Kd * (y - y_prev)
        uk = self.us + Pk + I + Dk
        uk = max(uk, 0)
        
        return uk, Ikp1


In [5]:
class TD2:
    def __init__(self, model ="EHM", **kwargs):
        self.model = model.upper()
        
        defaults = {} #tomt dictionary 
        with open('config.json', 'r') as f:
            data = json.load(f) #læs json-fil
        defaults.updata(data["general"]) #tilføj "general" til dictionary(defaults)
        defaults.update(data[self.model]) #tilføj modelegenskaberne til dictionary(defaults)
        defaults.update(kwargs) #tilføj keywordarguments til dictionary(defaults)

        if self.model == "MVP":
            self.f_func = lambda *args: MVP(self,*args) #caller MVP-modellen
        else:
            self.f_func = lambda *args: EHM(self,*args) #caller EHM-modellen

    def PID_controller(self, I, y, y_prev):
        """
        :input I: Integral term
        :input Gbar: Glucose concentration target
        :input y: Current blood glucose
        :input y_prev: Previous blood glucose
        :input us: Insulin steady state
        Tuning parameters
        :input Kp: Range 0-0.5
        :input Ti: 100-1000 minutes
        :input Td : 0-30 min
        """
        ek = y - self.Gbar
        Pk = self.Kp * ek
        Ki = self.Kp * self.timestep / self.Ti
        Ikp1 = I + Ki * ek
        Kd = self.Kp * self.Td / self.timestep
        Dk = Kd * (y - y_prev)
        uk = self.us + Pk + I + Dk
        uk = max(uk, 0)
        
        return uk, Ikp1
    
